## Install & Import Required Libraries

In [2]:
%pip install unsloth transformers huggingface_hub torch datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.

In [3]:
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
import torch
from trl import SFTTrainer
from datasets import load_dataset
from huggingface_hub import login
from transformers import TrainingArguments
import wandb
from google.colab import userdata
import os

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [7]:
hf_token = userdata.get('HF_TOKEN')
wandb_api_key = userdata.get('WANDB_API_KEY')

### Login to HF HUB & WandB.ai
login(hf_token)
# wandb.login(key=wandb_api_key)

# run = wandb.init(
#     project = "DeepSeek R1 finetuning for medical reasoning.",
#     job_type = "Fine-tuning",
#     anonymous= 'allow'
# )


## Now, We will load tokenizer and model

In [8]:
os.environ["UNSLOTH_USE_MODELSCOPE"] = "1"
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/DeepSeek-R1-Distill-Llama-8B',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

# Finetuning steps

### 1. Adding a system prompt

In [9]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

### 2. Download Medical Q&A Dataset from HF Hub

In [ ]:
### structure the train dataset to the training Prompt style.
def format_ds_prompt(examples):
  inputs = examples['Question']
  cots = examples['Complex_CoT']
  outputs = examples['Response']

  texts = []

  for input, cot, output in zip(inputs, cots, outputs):
    texts.append(train_prompt_style.format(input, cot, output) + tokenizer.eos_token)

  return {
      "text": texts
  }

dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split="train[:500]", trust_remote_code = True)
dataset_finetune = dataset.map(format_ds_prompt, batched=True)
dataset_finetune['text'][0]

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'FreedomIntelligence/medical-o1-reasoning-SFT' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'FreedomIntelligence/medical-o1-reasoning-SFT' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\nGiven the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?\n\n### Response:\n<think>\nOkay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is sw

### 3 Setting Up Model using QLoRA

In [12]:
## Basic Config for PEFT Model
r = 16
target_modules = ["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
bias = 'none'
lora_alpha = 32
lora_dropout = 0.0
use_gradient_checkpointing = 'unsloth'
random_state = 3407
use_rslora = False
loftq_config = None


model_lora = FastLanguageModel.get_peft_model(
    model,
    r=r,
    target_modules=target_modules,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias=bias,
    use_gradient_checkpointing=use_gradient_checkpointing,
    random_state=random_state,
    use_rslora=use_rslora,
    loftq_config=loftq_config
)

model_lora

Unsloth 2026.2.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

### 4. Using SFTTrainer to finetune model on our dataset.

In [13]:
# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_train_epochs = 1,
    warmup_steps = 5,
    max_steps = 60,
    learning_rate = 2e-4,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 10,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
)

# Define Trainer
trainer = SFTTrainer(
    model = model_lora,
    tokenizer = tokenizer,
    train_dataset = dataset_finetune,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = training_args,
)

Unsloth: Tokenizing ["text"] (num_proc=3):   0%|          | 0/500 [00:00<?, ? examples/s]

### 5. Model Training!

In [14]:
os.environ["UNSLOTH_TRAINING_LOW_MEMORY"] = "1"

train_state = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: Currently logged in as: alyannasir (alyannasir-codeffy-solutions) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
10,1.940400
20,1.380300
30,1.429400
40,1.421700
50,1.327400
60,1.478600


train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▄▄▄▁▅
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▂▂▁▃
total_flos,1958729139707904.0
train/epoch,0.12
train/global_step,60
train/grad_norm,1.23797
train/learning_rate,0.0
train/loss,1.4786


In [15]:
wandb.finish()

### 6. Save the Trained Model

In [18]:
adapter_save_path = "deepseek_r1_qlora_adapter"
model_lora.save_pretrained(adapter_save_path)
tokenizer.save_pretrained(adapter_save_path)
print(f"LoRA adapter saved to {adapter_save_path}")

LoRA adapter saved to deepseek_r1_qlora_adapter


### 7. Load the adapter for inference

In [23]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek_r1_qlora_adapter",
    max_seq_length = max_seq_length,
    dtype = dtype,
    token = hf_token
)

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [25]:
def ask_question(question, max_new_tokens=2048, temperature=0.7):
    inputs = tokenizer(question, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=0.9,
        repetition_penalty=1.05,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

question = "What are the main symptoms of hypertension?"
answer = ask_question(question)
print("Q:", question)
print("A:", answer)

Q: What are the main symptoms of hypertension?
A: What are the main symptoms of hypertension? High blood pressure, or hypertension, is a condition where the blood pressure is consistently high. It can lead to serious health issues if left untreated, such as heart disease, stroke, and kidney failure.

The primary symptom of hypertension is elevated blood pressure readings. This is usually measured through a sphygmomanometer, and it's classified as prehypertensive when readings are between 120/80 mmHg and 129/80 mmHg, and hypertensive when they reach 130/80 mmHg or higher. Other symptoms include headaches, dizziness, chest pain, palpitations, shortness of breath, and fatigue. It’s important to note that many people with hypertension don’t experience these symptoms, so regular blood pressure checks are crucial for early detection.

If you suspect you have hypertension, it’s wise to consult a healthcare professional for an accurate diagnosis and appropriate treatment plan. Early detection 